In [1]:
# import necessarry libraries

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib.request import urlopen

# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim 

# library to handle JSON files
import json

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

%matplotlib inline

# import k-means from clustering stage
from sklearn.cluster import KMeans

# map rendering library
import folium

# Requests Handler
import requests
from requests import get

ModuleNotFoundError: No module named 'folium'

In [2]:
# Web scrape the wiki page using beautiful soup so that we can load the table into a dataframe

wiki_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
wiki_page = urlopen(wiki_url)
b_soup = BeautifulSoup(wiki_page,'html.parser')

In [3]:
# Find the table

wiki_table = b_soup.body.table.tbody

# Create a dataframe to store the table

table_df = pd.DataFrame(columns=['Postal Code','Borough','Neighborhood'])

In [4]:
# Extract the table values and append into dataframe

for tr_cell in wiki_table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        table_df.loc[len(table_df)] = row_data

In [5]:
# Check initial dataframe shape

table_df.shape

(180, 3)

In [6]:
# remove rows where Borough is 'Not assigned'

table_df=table_df[table_df['Borough']!='Not assigned']

In [7]:

# assign Neighbourhood=Borough where Neighbourhood is 'Not assigned'

table_df[table_df['Neighborhood']=='Not assigned']=table_df['Borough']

ValueError: cannot set using a list-like indexer with a different length than the value

In [8]:
# Check the Table

table_df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [9]:
# group multiple Neighbourhood under one Postcode

temp_df=table_df.groupby('Postal Code')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)

In [10]:
# join the newly constructed joined data frame

df_merged = pd.merge(table_df, temp_df, on='Postal Code')

In [11]:
# drop the Neighbourhood column

df_merged.drop(['Neighborhood'],axis=1,inplace=True)

In [12]:
# drop duplicates from the data frame

df_merged.drop_duplicates(inplace=True)

In [13]:
# rename Neighbourhood_joined back to Neighbourhood

df_merged.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)
df_merged.head()


,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [14]:

# The postalcode for Queen's park is missing and has to be replaced with its code M7A

df_merged["Postal Code"].replace(to_replace="Queen's Park", value="M7A", inplace=True)
df_merged.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [15]:

# Since it takes long time to read from the geocode package we will use the csv file provided

geo_df=pd.read_csv('http://cocl.us/Geospatial_data')
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
# Before we merge the Neighborhood data frame with the Geospatial dataframe we need to sort the Neighborhood data frame by the Postal Code first

df_merged_sorted = df_merged.sort_values("Postal Code")
df_merged_sorted.head()

,Postal Code,Borough,Neighborhood
6,M1B,Scarborough,"Malvern, Rouge"
12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
18,M1E,Scarborough,"Guildwood, Morningside, West Hill"
22,M1G,Scarborough,Woburn
26,M1H,Scarborough,Cedarbrae


In [17]:
# Merge the two dataframes

geo_merged_df = pd.merge(df_merged_sorted, geo_df, on='Postal Code')
geo_merged_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [18]:
# work with only boroughs that contain the word Toronto

toronto_df=geo_merged_df[geo_merged_df['Borough'].str.contains("Toronto")]
toronto_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [19]:
city = 'Toronto'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [22]:
# Define FourSquare Credentials

CLIENT_ID = '3J3TP0GCUC5WHV1MJLUFJPZNZHCH35UXJIKEVTCWZL44HJT0' # your Foursquare ID
CLIENT_SECRET = 'OHKJDQFULROBSBQVJEVAD1OC51VEGWVTTOCOHRS5ONIWTB2M' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
radius=500
LIMIT=100

In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [25]:
# Load the above result into a new data frame

toronto_venues = getNearbyVenues(names=toronto_df['Neighborhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )

The Beaches


NameError: name 'requests' is not defined

In [26]:

toronto_venues.head()

NameError: name 'toronto_venues' is not defined